# Forecasting Strategies (Recursive, Direct, Multioutput)

Multi-step forecasting depends on *how* you roll a model forward over a horizon.
This notebook compares the main strategy families used in sktime-style forecasting.


## Notation refresher

Let $y_t$ be the observed series. For a forecast origin $t_0$ and horizon $h$:

\[\hat{y}_{t_0 + k\mid t_0}, \; k=1,\dots,h\]

Different strategies decide **how to generate** the vector
$\hat{\mathbf{y}}_{t_0+1:t_0+h}$ and what training data to reuse.


## Strategy families

- **Recursive (iterated)**: fit a 1-step model and feed predictions back in.
- **Direct**: train **one model per horizon step** $k$.
- **Multioutput**: train **one model** to output the full horizon vector.
- **DirRec (hybrid)**: mix direct targets with recursive inputs.

**Trade-offs** (intuition):
- Recursive is data-efficient but error can compound.
- Direct avoids compounding but costs more models.
- Multioutput shares info across steps but can be harder to fit.


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

np.random.seed(7)

# Synthetic seasonal series
n = 180
t = np.arange(n)
y = 0.05 * t + 2.0 * np.sin(2 * np.pi * t / 24) + np.random.normal(0, 0.6, n)

t0 = 140
h = 16
fh = np.arange(1, h + 1)

train_idx = np.arange(t0)
forecast_idx = t0 + fh

# Simple mock strategies
last_value = y[t0 - 1]
recursive_pred = np.full(h, last_value)

# Direct: linear extrapolation from last 24 points
coef = np.polyfit(t[t0-24:t0], y[t0-24:t0], 1)
direct_pred = coef[0] * forecast_idx + coef[1]

# Multioutput: moving average + seasonal continuation
seasonal = 2.0 * np.sin(2 * np.pi * forecast_idx / 24)
ma = np.mean(y[t0-12:t0])
multi_pred = ma + seasonal

preds = {
    "Recursive": recursive_pred,
    "Direct": direct_pred,
    "Multioutput": multi_pred,
}

fig = make_subplots(rows=1, cols=3, shared_yaxes=True, subplot_titles=list(preds.keys()))

for col, (name, pred) in enumerate(preds.items(), start=1):
    fig.add_trace(
        go.Scatter(x=t, y=y, mode="lines", name="Series", line=dict(color="#2a3f5f")),
        row=1,
        col=col,
    )
    fig.add_trace(
        go.Scatter(x=forecast_idx, y=pred, mode="markers+lines", name="Forecast", line=dict(color="#ef553b")),
        row=1,
        col=col,
    )
    fig.add_vline(x=t0, line_dash="dash", line_color="#636efa", row=1, col=col)

fig.update_layout(
    height=420,
    width=1100,
    showlegend=False,
    title="Forecasting strategies at the same origin",
)
fig.show()


## sktime mapping (practical pointers)

In sktime, the **forecasting horizon** is typically passed as `fh` and determines
which $k$ values are predicted. Reduction strategies are available via utilities
like `make_reduction` (API varies by version) with strategy choices such as
*recursive*, *direct*, or *multioutput*.

**Recommended workflow**:
1. Pick a strategy based on horizon length and data volume.
2. Validate using backtesting (see the backtesting notebook).
3. Compare with probabilistic intervals for risk-aware decisions.
